In [60]:
using ImmersedLayers

In [61]:
using Revise

In [62]:
using Plots

In [63]:
using Interpolations

In [64]:
using UnPack

In [65]:
abstract type AbstractGrids end
struct constructGrids{XT,CT} <: AbstractGrids

    "Grid spacing"
    Δx::Float64

    "size of domain in x-direction"
    xlim::XT

    "size of domain in y-direction"
    ylim::XT
    
    "Physical Grid"
    g::PhysicalGrid

    "Cache"
    cache::CT

    "Number of points on heater boundary"
    Ntheta::Int64
end

function constructGrids(Δx,bounds;Ntheta=500)
    xlim = bounds[1]
    ylim = bounds[2]
    g = PhysicalGrid(xlim,ylim,Δx)
    cache = SurfaceScalarCache(g)
    CacheType = typeof(cache)
    constructGrids{typeof(xlim), CacheType}(Δx,xlim,ylim,g,cache,Ntheta)
end

constructGrids

In [66]:
function create_points_on_shape(x::AbstractVector,gridConfig::constructGrids)
    @unpack Ntheta = gridConfig
    theta = collect(range(0,2π,Ntheta))
    c0 = x[1] + im*x[2]
    q = x[3]
    c1 = x[4]
    c2 = x[5]
    r = [c0 + c1*exp(1im*theta[i]) + c2*exp(2im*theta[i]) for i in 1:Ntheta]
    interpolator_real = linear_interpolation(theta,real(r))
    interpolator_imag = linear_interpolation(theta,imag(r))
    return interpolator_real, interpolator_imag
end

create_points_on_shape (generic function with 1 method)

In [67]:
struct HeaterConfig{SID} 

    "Number of heaters"
    Nq::Int64

    "State IDs"
    state_id::SID

    "Time step"
    Δt::Float64

end

function HeaterConfig(Nq,Δt)
  state_id = construct_heater_state_mapping(Nq)
  HeaterConfig{typeof(state_id)}(Nq,state_id,Δt)
end

function HeaterConfig(Nq)
  state_id = construct_heater_state_mapping(Nq)
  HeaterConfig{typeof(state_id)}(Nq,state_id,0.0)
end

HeaterConfig

In [68]:
function construct_heater_state_mapping(Nq::Int64)
    state_id = Dict()
    heater_x_ids = zeros(Int,Nq)
    heater_y_ids = zeros(Int,Nq)
    heater_q_ids = zeros(Int,Nq)
    heater_c1_ids = zeros(Int,Nq)
    heater_c2_ids = zeros(Int,Nq)
  
    for j in 1:Nq
      heater_x_ids[j] = 5j-4
      heater_y_ids[j] = 5j-3
      heater_q_ids[j] = 5j-2
      heater_c1_ids[j] = 5j-1
      heater_c2_ids[j] = 5j
    end
  
    state_id["heater x"] = heater_x_ids
    state_id["heater y"] = heater_y_ids
    state_id["heater q"] = heater_q_ids
    state_id["heater c1"] = heater_c1_ids
    state_id["heater c2"] = heater_c2_ids
  
    return state_id
  end

construct_heater_state_mapping (generic function with 1 method)

In [69]:
function positions_and_strengths_to_state(zq::AbstractVector{ComplexF64},qq::AbstractVector{Float64},c1q::AbstractVector{Float64},c2q::AbstractVector{Float64},config::HeaterConfig)
    @unpack Nq, state_id = config
    state = zeros(state_length(state_id))
  
    x_ids = state_id["heater x"]
    y_ids = state_id["heater y"]
    q_ids = state_id["heater q"]
    c1_ids = state_id["heater c1"]
    c2_ids = state_id["heater c2"]
  
  
    for i = 1:Nq
      state[x_ids[i]] = real(zq[i])
      state[y_ids[i]] = imag(zq[i])
      state[q_ids[i]] = qq[i]
      state[c1_ids[i]] = c1q[i]
      state[c2_ids[i]] = c2q[i]
    end
  
    return state
  end

positions_and_strengths_to_state (generic function with 1 method)

In [70]:
abstract type AbstractConfig end
state_length(config::AbstractConfig) =  state_length(config.state_id)

state_length(a::Dict) = mapreduce(key -> state_length(a[key]),+,keys(a))
state_length(a::Vector) = length(a)
state_length(a::Matrix) = 0
state_length(a::Int) = 0

state_length (generic function with 5 methods)

In [71]:
function get_config_and_state(zq::AbstractVector,qq::AbstractVector,c1q::AbstractVector,c2q::AbstractVector)
    Nq = length(zq)
    config = HeaterConfig(Nq)
    x = positions_and_strengths_to_state(zq,qq,c1q,c2q,config)
  
    return config, x
  end

get_config_and_state (generic function with 1 method)

In [72]:
mutable struct TemperatureObservation{Nx,Ny,ST,CT}
	sens::ST
	config::CT
end
function TemperatureObservation(sens::AbstractVector,config::HeaterConfig)
    return TemperatureObservation{5*config.Nq,length(sens),typeof(sens),typeof(config)}(sens,config)
end
function observations(x::AbstractVector,t,obs::TemperatureObservation,gridConfig::constructGrids)
    return analytical_temperature(x,obs,gridConfig)
  end

observations (generic function with 1 method)

In [73]:
function setup_sensors(Nsens;layout=(:line,1.0))

    layout_type, len = layout
  
    if layout_type == :circle
      rsens = len
      θsens = range(0,2π,length=Nsens+1)
      sens = rsens*exp.(im*θsens[1:end-1])
    elseif layout_type == :line
      ϵsens = 0.0
      lowerrow = range(-len,len,length=Nsens) .+ (-0.5ϵsens .+ ϵsens*rand(Nsens))*im
      #upperrow = range(-2.0,2.0,length=Nsens) .+ 1.0*im
      #leftside = im*range(-1.0,3.0,length=Nsens) .- 1.0
      #rightside = im*range(-1.0,3.0,length=Nsens) .+ 1.0
      sens = vcat(lowerrow,)  #upperrow);
    elseif layout_type == :dline
      ϵsens = 0.02
      lowerrow1 = range(-len,len,length=Nsens÷2) .- 0.5*ϵsens
      lowerrow2 = range(-len,len,length=Nsens÷2) .+ 0.5*ϵsens
      sens = sort(vcat(lowerrow1,lowerrow2)) .+ 0.0im
    end
    return sens
  end

setup_sensors (generic function with 1 method)

In [74]:
function analytical_temperature(x::AbstractVector,obs::TemperatureObservation,gridConfig::constructGrids)
	@unpack config, sens = obs
	@unpack Nq, state_id = config
	@unpack g, cache, Ntheta = gridConfig
	Ny = length(sens)

	x_ids = state_id["heater x"]
  	y_ids = state_id["heater y"]
  	q_ids = state_id["heater q"]
  	c1_ids = state_id["heater c1"]
	c2_ids = state_id["heater c2"]

	xq = x[x_ids]
	yq = x[y_ids]
	qq = x[q_ids]
	c1q = x[c1_ids]
	c2q = x[c2_ids]

	T = zeros_grid(cache)
	xg, yg = coordinates(T,g)
	Temp = zeros(Ny)

	for k in 1:Nq
		r_real, r_imag = create_points_on_shape(x[5k-4:5k],gridConfig)
		T = zeros_grid(cache)
		theta_g = zeros_grid(cache)
		for (j,yy) in enumerate(yg), (i,xx) in enumerate(xg)
			theta_g[i,j] = (yy-yq[k])>=0 ? atan((yy-yq[k]),(xx-xq[k])) : (atan((yy-yq[k]),(xx-xq[k]))+2π)
			if ((xx-xq[k])^2+(yy-yq[k])^2) < ((r_real(theta_g[i,j])-xq[k])^2 + (r_imag(theta_g[i,j])-yq[k])^2)
				T[i,j] = -qq[k]
			end
		end
		inverse_laplacian!(T,cache)
		Tfield = interpolatable_field(T,g)
		Temp .+= [Tfield(real(sens[j]), imag(sens[j])) for j in 1:Ny]
	end
	return Temp
end

analytical_temperature (generic function with 1 method)

In [75]:
#zq = [0.5+1.2im,-0.5+0.7im]
#qq = [1.0,2.0]
#c1q = [0.5,0.2]
#c2q = [0.0, 0.0]
#Nq_true = length(zq)

#zq = [1.0+0.0im]
#qq = [1.0]
#c1q = [0.5]
#c2q = [0.25]
#Nq_true = length(zq)

zq = [0.5+1.2im]
qq = [1.0]
c1q = [0.5]
c2q = [0.0]
Nq_true = length(zq)


1

In [76]:
config_true, x_true = get_config_and_state(zq,qq,c1q,c2q)
x_true

5-element Vector{Float64}:
 0.5
 1.2
 1.0
 0.5
 0.0

In [77]:
Ntheta = 500

500

In [78]:
xr = (-2.0,2.0)
yr = (-2,2.0) #(0.01,4) #(0.01,2.0)
qr = (-2.0,2.0)
c1r = (0.0,1.0)
c2r = (0.0,1.0)
bound = [xr,yr,qr,c1r,c2r]


5-element Vector{Tuple{Real, Float64}}:
 (-2.0, 2.0)
 (-2, 2.0)
 (-2.0, 2.0)
 (0.0, 1.0)
 (0.0, 1.0)

In [79]:
Δx = 0.01
gridConfig = constructGrids(Δx,bound;Ntheta)

constructGrids{Tuple{Float64, Float64}, BasicILMCache{0, GridScaling, Nodes{Primal, 402, 408, Float64, Matrix{Float64}}, 2, BodyList, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Regularize{0, false}, RegularizationMatrix{Edges{Primal, 402, 408, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Edges{Primal, 402, 408, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, RegularizationMatrix{Nodes{Primal, 402, 408, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Nodes{Primal, 402, 408, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, Nothing, Nothing, Nothing, Nothing, Laplacian{402, 408, Float64, true, false}, Edges{Primal, 402, 408, Float64, Vector{Float64}}, Nodes{Dual, 402, 408, Float64, Matrix{Float64}}, Nothing, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Nothing}}(0.01, (-2.0, 2.0), (-2.0, 2.

In [80]:
Nsens = 3
sens = setup_sensors(Nsens;layout=(:line,1.0))

3-element Vector{ComplexF64}:
 -1.0 + 0.0im
  0.0 + 0.0im
  1.0 + 0.0im

In [81]:
obs_true = TemperatureObservation(sens,config_true)

TemperatureObservation{5, 3, Vector{ComplexF64}, HeaterConfig{Dict{Any, Any}}}(ComplexF64[-1.0 + 0.0im, 0.0 + 0.0im, 1.0 + 0.0im], HeaterConfig{Dict{Any, Any}}(1, Dict{Any, Any}("heater q" => [3], "heater y" => [2], "heater x" => [1], "heater c1" => [4], "heater c2" => [5]), 0.0))

In [82]:
x_true

5-element Vector{Float64}:
 0.5
 1.2
 1.0
 0.5
 0.0

In [83]:
t = 0.0
ystar0 = observations(x_true,t,obs_true,gridConfig)

3-element Vector{Float64}:
 -0.654822624815675
 -0.6061965993715731
 -0.606196599371573